In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('../data/graph_train.csv')
y = pd.cut(df['edge_cross_norm'], bins=[-1,-1e-10,1]).to_numpy()
df = df.drop(labels=['edge_cross_norm','edge_id','graph_id','num_nodes','num_edges','benchmark','max_deg','min_deg','Unnamed: 0'],axis=1)
list_columns = list(df)

for col in list_columns:
    print(col)
    if col!= 'edge_cross_norm' and col!= 'is_bridge':
        df[col] = pd.qcut(df[col],q=5)

X = df.to_numpy()
df.head()


edge_betweenness
stress
is_bridge
diff_stress
diff_cross
diff_edgelength
exp_factor_norm
sum_neighbour_deg_norm
max_neighbour_deg_norm
grad_diff


,edge_betweenness,stress,is_bridge,diff_stress,diff_cross,diff_edgelength,exp_factor_norm,sum_neighbour_deg_norm,max_neighbour_deg_norm,grad_diff
0,"(0.0451, 0.0702]","(0.0818, 0.684]",False,"(0.357, 1.41]","(-2.0, 0.0]","(-0.00824, -0.00378]","(0.0607, 0.109]","(0.158, 1.64]","(0.107, 0.92]","(0.211, 7.47]"
1,"(-0.000785, 0.0219]","(0.0818, 0.684]",False,"(-0.0453, 0.357]","(-2.0, 0.0]","(-0.00146, 0.00165]","(-0.0009714, 0.0192]","(0.158, 1.64]","(0.107, 0.92]","(0.211, 7.47]"
2,"(0.0316, 0.0451]","(0.0818, 0.684]",False,"(-0.0453, 0.357]","(-2.0, 0.0]","(-0.00146, 0.00165]","(-0.0009714, 0.0192]","(0.158, 1.64]","(0.107, 0.92]","(0.211, 7.47]"
3,"(0.0316, 0.0451]","(0.0818, 0.684]",False,"(-1.163, -0.0453]","(2.0, 39.0]","(-0.516, -0.00824]","(0.109, 0.607]","(0.158, 1.64]","(0.107, 0.92]","(0.211, 7.47]"
4,"(0.0702, 0.545]","(0.0818, 0.684]",False,"(-1.163, -0.0453]","(-2.0, 0.0]","(0.00165, 0.266]","(0.0607, 0.109]","(0.158, 1.64]","(0.107, 0.92]","(0.211, 7.47]"


In [39]:
X_train,X_test, y_train, y_test = train_test_split(X,y)

In [40]:
y[0]

Interval(-1e-10, 1.0, closed='right')

In [57]:
# XGBoost prep
import numpy as np
df2 = pd.read_csv('../data/graph_train.csv')
df2['is_bridge'] = df2['is_bridge'].astype(float)
yn = df2['edge_cross_norm'].to_numpy(dtype=np.float64)
df2 = df2.drop(labels=['edge_cross_norm','edge_id','graph_id','num_nodes','num_edges','benchmark','max_deg','min_deg','Unnamed: 0'],axis=1)
Xn = df2.to_numpy(dtype=np.float64)

Xn_train, Xn_test, yn_train, yn_test = train_test_split(Xn,yn)

In [59]:
yn[0]

0.0

In [61]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)

In [62]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb.fit(Xn_train,yn_train, eval_metric = 'auc')
print(xgb)
y_pred = xgb.predict(X_test)

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [   0    1    2 ... 2369 2370 2371], got [-0.125      -0.06060606 -0.05833333 ...  0.02371542  0.02380952
  0.03333333]